In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

import sys
import os

sys.path.append(os.path.relpath("../../huygens"))
sys.path.append(os.path.relpath("../../galileo"))

import galileo as gal
import huygens as huy

# Load annotations

In [9]:
ccle_genex = pd.read_hdf(
    "../../data/processed/ccle/CCLE_RNAseq_rsem_genes_tpm_20180929.hdf", key="ccle_genex")
ccle_transcripts = pd.read_hdf("../../data/processed/ccle/CCLE_RNAseq_rsem_transcripts_tpm_20180929.hdf",
                        key="ccle_transcripts")
exonusage = pd.read_hdf(
    "../../data/processed/ccle/CCLE_RNAseq_ExonUsageRatio_20180929.hdf", key="exonusage")

ms_prot = pd.read_hdf("../../data/processed/ccle/ms_prot.h5",key="ms_prot")
rppa = pd.read_hdf("../../data/processed/ccle/CCLE_RPPA_20181003.hdf",key="rppa")

In [32]:
msi = pd.read_excel("../data/external/ccle2/41586_2019_1186_MOESM10_ESM.xlsx",sheet_name="MSI calls")
msi = msi.set_index("depMapID")
msi["GDSC_msi"] = msi["GDSC.msi.call"].replace({"MSI-H":True,"MSS/MSI-L":False})
msi["CCLE_msi"] = msi["CCLE.MSI.call"].replace({"inferred-MSI":True,"inferred-MSS":False,"undetermined":np.nan})

msi = msi.dropna(subset=["GDSC_msi","CCLE_msi"],how="all")

msi["MSI"] = msi["GDSC_msi"] | msi["CCLE_msi"]
name_map = dict(zip(msi["CCLE_ID"],msi.index))
name_map["HS294T_SKIN"] = "ACH-000014"

# Compute differences

In [ ]:
msi_prot_diffs = gal.mat_mwus_naive(ms_prot,msi["MSI"],pbar=True)
msi_prot_diffs.to_csv("../data/intermediate/msi_prot_diffs.txt",sep="\t")

msi_exon_diffs = gal.mat_mwus_naive(exonusage,msi["MSI"],pbar=True,effect="mean")
msi_exon_diffs.to_csv("../data/intermediate/msi_exon_diffs.txt",sep="\t")

# Mutations

In [2]:
msi_prot_diffs = pd.read_csv("../data/intermediate/msi_prot_diffs.txt",sep="\t")
msi_exon_diffs = pd.read_csv("../data/intermediate/msi_exon_diffs.txt",sep="\t",index_col=0)

## SRA info

In [33]:
ccle_wgs_sra = pd.read_csv("../data/raw/SRA_info/CCLE_WGS_accessions.txt")
ccle_wgs_sra["ach_id"] = ccle_wgs_sra["Sample Name"].apply(lambda x: name_map[x])

## Gene intervals

In [15]:
g19_definitions = pd.read_hdf("../../data/processed/ccle/gencode.v19.genes.v7_model.patched_contigs.h5",
                       key="g19_definitions")

g19_genes = g19_definitions[g19_definitions["feature"]=="gene"]

g19_genes = g19_genes.set_index("gene_id")

In [76]:
msi_exons = msi_exon_diffs.copy()[msi_exon_diffs["qval"]>4]

msi_exons["gene"] = msi_exons.index.map(lambda x: x.split("_")[-1])
msi_exons["exon_chrom"] = msi_exons.index.map(lambda x: x.split("_")[-4])
msi_exons["exon_start"] = msi_exons.index.map(lambda x: x.split("_")[-3])
msi_exons["exon_end"] = msi_exons.index.map(lambda x: x.split("_")[-2])

msi_exons["exon"] = msi_exons["exon_chrom"] + "_" + msi_exons["exon_start"] + "_" + msi_exons["exon_end"]

msi_exons["exon_start"] = msi_exons["exon_start"].astype(int)
msi_exons["exon_end"] = msi_exons["exon_end"].astype(int)

msi_exons = msi_exons.drop_duplicates(subset=["exon"])

In [84]:
def get_exon_slices(row,padding=1000):
    if row["exon_start"] < row["exon_end"]:
        return row["exon_chrom"][3:]+":"+str(row["exon_start"]-padding) + "-" + str(row["exon_end"]+padding)
    
    elif row["exon_start"] > row["exon_end"]:
        return row["exon_chrom"][3:]+":"+str(row["exon_end"]-padding) + "-" + str(row["exon_start"]+padding)
    
exon_slices = msi_exons.apply(get_exon_slices,axis=1)

In [89]:
# msi_genes = g19_genes.loc[msi_genes]
# msi_genes = msi_genes.sort_values(by=["seqname", "start"])

# msi_genes["filter_region"] = msi_genes["seqname"].astype(
#     str) + ":" + msi_genes["start"].astype(str) + "-" + msi_genes["end"].astype(str)

regions_1 = ",".join(exon_slices[:250])
regions_2 = ",".join(exon_slices[250:])

with open("../scripts/7_fetch-msi-slices.sh", "w") as f:
    for sra_id, ach_id in zip(list(ccle_wgs_sra["Run"]), list(ccle_wgs_sra["ach_id"])):

        aligned_arg = "--aligned-region " + regions_1 + " "
        output_arg = "--outfile ../data/raw/WGS_slices/" + ach_id + "_1.sam "

        f.write("sra-pileup " + aligned_arg + output_arg + sra_id + "\n")
        
        aligned_arg = "--aligned-region " + regions_2 + " "
        output_arg = "--outfile ../data/raw/WGS_slices/" + ach_id + "_2.sam "

        f.write("sra-pileup " + aligned_arg + output_arg + sra_id + "\n")